## Importing Libraries 

In [15]:
from flask import Flask
import pandas as pd
import random
from flask import Flask, request, jsonify
import numpy as np
from IPython.display import HTML, display

## Importing Dataset

In [16]:
weather = pd.read_csv('weatherDataset.csv')
aircraft = pd.read_csv('aircraftDataset.csv')
airport = pd.read_csv('airportDataset.csv')

In [17]:
def fitnessFunctionAircraft(aircraft_data, weights=(0.4, 0.3, 0.3)):
    fitness_dict = {} 
    max_range_values = aircraft_data['Max Range']
    cruise_speed_values = aircraft_data['Cruise Speed']
    fuel_consumption_values = aircraft_data['Fuel Consumption at Cruise']
    icao_codes = aircraft_data['ICAO CODES']
    aircrafts = aircraft_data['Aircraft Type']

    for i in range(len(aircraft_data)):
        max_range_score = max_range_values[i]
        cruise_speed_score = cruise_speed_values[i]
        fuel_consumption_score = 1 / fuel_consumption_values[i]
        fitness_score = (
            weights[0] * max_range_score +
            weights[1] * cruise_speed_score +
            weights[2] * fuel_consumption_score
        )
        fitness_dict[fitness_score] = (icao_codes[i],aircrafts[i])

    return fitness_dict

In [18]:
def selection(listData, k_value):
    listData = listData[:k_value]
    return listData

In [19]:
def generateNewPath(result_list , newPath , date):
    results = []
    population = []
    for i in range(len(result_list)):
        for index, row in airport[airport['ICAO Code'] == newPath].iterrows():
            icao_code = row['ICAO Code']
            city = row['City']
            latitude = row['Latitude']
            longitude = row['Longitude']
            travelTime = latitude - longitude / result_list[i][1]
            results.append((icao_code, city, result_list[i][0] , travelTime))

    sorted_result = sorted(results, key=lambda x: x[-1])

    for i in range(len(sorted_result)):
        if (i == 0) or (i % 5 == 0):
            population.append(sorted_result[i])

    population = selection(population , 5)
        
    weather['Wind Impact'] = weather['Wind Speed'] * np.sin(np.deg2rad(weather['Wind Direction']))
    weather['Safe/Unsafe'] = weather['Wind Impact'].apply(lambda x: 0 if 20 <= x <= 30 else 1)
    weatherWithdate =  weather[weather['Date'] == date]
    
    city_safe_mapping = dict(zip(weatherWithdate['City'], weatherWithdate['Safe/Unsafe']))
    safeCities = []
    for _, city, _, _ in population:
        if city in city_safe_mapping:
            safe_unsafe = city_safe_mapping[city]
            safeCities.append(city)
    return safeCities
    

In [20]:
def GeneticAlgorithm(source, destination, date, optimizedPath=None):
    if optimizedPath is None:
        optimizedPath = [source]

    fitnessValues = fitnessFunctionAircraft(aircraft, weights=(0.4, 0.3, 0.3))
    sourceAircrafts = [(key, value[1]) for key, value in fitnessValues.items() if value[0] == source]
    sortedSourceAircrafts = sorted(sourceAircrafts, key=lambda x: x[0], reverse=True)
    bestPopulation = selection(sortedSourceAircrafts, 5)
    result_list = []
    for i in bestPopulation:
        aircraft_type = i[1]
        cruise_speed = aircraft['Cruise Speed'][aircraft['Aircraft Type'] == aircraft_type].values[0]
        result_list.append((aircraft_type, cruise_speed))

    safeCities = generateNewPath(result_list, source , date)
    newPATHS = []

    for i in safeCities:
        i = i[:4].upper()
        if destination == i:
            optimizedPath.append(destination)
            return optimizedPath
        else:
            newPATHS.append(i)

    optimizedPath.append(newPATHS[0])

    return GeneticAlgorithm(newPATHS[0], destination, date, optimizedPath)
    

In [21]:
listP = GeneticAlgorithm('ISLA','GWAD','2007-01-01')
listP

['ISLA', 'QUET', 'GUJR', 'GWAD']

## Flask

In [ ]:
app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def hello():
    optimizedPath = None
    source = "" 
    destination = "" 
    date = ""  

    if request.method == 'POST':
        source = request.form.get('source')
        destination = request.form.get('destination')
        date = request.form.get('date')
        optimizedPath = GeneticAlgorithm(source, destination, date)

    html = f'''
    <html>
    <head>
        <style>
        @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap');

        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Montserrat', sans-serif;
        }}

        body {{
            display: flex;
            align-items: center;
            justify-content: center;
            flex-direction: column;
            height: 100vh;
        }}

        .container {{
            background-image: url('sa.png');
            background-size: cover;
            background-repeat: no-repeat;
            border-radius: 30px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.35);
            position: relative;
            overflow: hidden;
            width: 768px;
            max-width: 100%;
            min-height: 480px;
        }}

        .container p{{
            font-size: 14px;
            line-height: 20px;
            letter-spacing: 0.3px;
            margin: 20px 0;
        }}

        .container span{{
            font-size: 12px;
        }}

        .container a{{
            color: #333;
            font-size: 13px;
            text-decoration: none;
            margin: 15px 0 10px;
        }}

        .container button{{
            background-color: #1ac0a1;
            color: #fff;
            font-size: 12px;
            padding: 10px 45px;
            border: 1px solid transparent;
            border-radius: 8px;
            font-weight: 600;
            letter-spacing: 0.5px;
            text-transform: uppercase;
            margin-top: 10px;
            cursor: pointer;
        }}

        .container button.hidden{{
            background-color: transparent;
            border-color: #fff;
        }}

        .container form{{
            background-color: #fff;
            display: flex;
            align-items: center;
            justify-content: center;
            flex-direction: column;
            padding: 0 40px;
            height: 100%;
        }}

        .container input{{
            background-color: #eee;
            border: none;
            margin: 8px 0;
            padding: 10px 15px;
            font-size: 13px;
            border-radius: 8px;
            width: 100%;
            outline: none;
        }}

        .form-container{{
            position: absolute;
            top: 0;
            height: 100%;
            transition: all 0.6s ease-in-out;
        }}


        .container.active .sign-in{{
            transform: translateX(100%);
        }}


        .container.active .sign-up{{
            transform: translateX(100%);
            opacity: 1;
            z-index: 5;
            animation: move 0.6s;
        }}
        </style>
    </head>
    <body>
        <h1>Aircraft Route Optimization with Genetic Algorithm!</h1><br>
        <div class="container" id="container">
            <form method="POST">
                <input type="text" name="source" placeholder="Source">
                <br><input type="text" name="destination" placeholder="Destination">
                <br><input type="text" name="date" placeholder="Date - DDD-MMM-YYY">
                <button type="submit">Search</button>
            </form>
        </div>
        <h2>Optimized Path:</h2>
        <p>{optimizedPath}</p>  
    </body>
    </html>
    '''
    return html



if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Nov/2023 00:55:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2023 00:55:36] "GET /sa.png HTTP/1.1" 404 -
[2023-11-02 00:55:43,545] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/Users/AbdulRafay/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/AbdulRafay/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/AbdulRafay/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/AbdulRafay/opt/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/AbdulRafay/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = se